In [ ]:
# Importação

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pycountry

In [ ]:
# lendo a base de dados

df = pd.read_csv('../data/processed/salarios.csv')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df['residencia_iso3'].value_counts()

In [ ]:
# Plotando com pandas a Distribuição do Nível de Experiência em um gráfico de barras

ax = df['nivel_experiencia'].value_counts().plot(kind='bar', title='Distribuição do Nível de Experiência')

for p in ax.patches:
    ax.annotate(f'{p.get_height():.0f}',
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='bottom')

In [ ]:
# Plotando com seaborn o salário_dolar por Nível de Experiência em um gráfico de barras

# sns.set(style="whitegrid")
ax = sns.barplot(data=df, x='nivel_experiencia', y='salario_dolar')
ax.set_title('Distribuição do Salário médio em dolar por Nível de Experiência')

for p in ax.patches:
    ax.annotate(f'{p.get_height():.0f}',
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='bottom')

plt.show()

In [ ]:
# Personalizando o gráfico de barras do salário_dolar por Nível de Experiência

plt.figure(figsize=(8,5))
ax = sns.barplot(data=df, x='nivel_experiencia', y='salario_dolar')
plt.title("Salário médio por senioridade")
plt.xlabel("Nível de Experiência")
plt.ylabel("Salário médio anual (USD)")

# explicitando as legendas dos dados
for p in ax.patches:
    ax.annotate(
        f'{p.get_height():.0f}',
        (p.get_x() + p.get_width() / 2., p.get_height()),
        ha='center', va='bottom'
    )

plt.show()

In [ ]:
# agrupando a média do salario_dolar por nível_experiencia

df.groupby('nivel_experiencia')['salario_dolar'].mean().sort_values(ascending=False)

In [ ]:
# Plotando o gráfico de barras do salário_dolar por Nível de Experiência ordenado por Nível de Experiência(média)

nivel_ordenado = df.groupby('nivel_experiencia')['salario_dolar'].mean().sort_values(ascending=True).index

plt.figure(figsize=(8,5))
ax = sns.barplot(data=df, x='nivel_experiencia', y='salario_dolar', order=nivel_ordenado)
plt.title("Salário médio por senioridade")
plt.xlabel("Senioridade")
plt.ylabel("Salário médio anual (USD)")

# explicitando as legendas dos dados
for p in ax.patches:
    ax.annotate(
        f'{p.get_height():.0f}',
        (p.get_x() + p.get_width() / 2., p.get_height()),
        ha='center', va='bottom'
    )

plt.show()

In [ ]:
plt.figure(figsize=(10,5))
ax = sns.histplot(df['salario_dolar'], bins = 50, kde=True)
plt.title("Distribuição dos salários anuais")
plt.xlabel("Salário em USD")
plt.ylabel("Frequência")
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(x=df['salario_dolar'])
plt.title("Boxplot Salário")
plt.xlabel("Salário em USD")
plt.show()

In [ ]:
df.columns

In [ ]:
ordem_senioridade = ['Junior', 'Pleno', 'Senior', 'Executivo']

plt.figure(figsize=(8,5))
sns.boxplot(x='nivel_experiencia', y='salario_dolar', data=df, order=ordem_senioridade)
plt.title("Boxplot da distribuição do Salário em Dolar por Nível de Experiência")
plt.xlabel("Nível de Experiência")
plt.ylabel("Salário em Dolar")
plt.show()

In [ ]:
ordem_senioridade = ['Junior', 'Pleno', 'Senior', 'Executivo']

plt.figure(figsize=(8,5))
sns.boxplot(x='nivel_experiencia', y='salario_dolar', data=df, order=ordem_senioridade, palette='Set2', hue='nivel_experiencia')
plt.title("Boxplot da distribuição do Salário em Dolar por Nível de Experiência")
plt.xlabel("Nível de Experiência")
plt.ylabel("Salário em Dolar")
plt.show()

In [ ]:
# gráfico de média salarial por senioridade em barras usando o plotly

ax = df.groupby('nivel_experiencia')['salario_dolar'].mean().sort_values(ascending=False).reset_index()

fig = px.bar(ax,
             x='nivel_experiencia',
             y='salario_dolar',
             title='Média Salarial por Nível de Experiência',
             labels={'nivel_experiencia': 'Nível de Senioridade', 'salario_dolar': 'Média Salarial Anual (USD)'})

fig.show()

In [ ]:
remoto_contagem = df['regime_trabalho'].value_counts().reset_index()
remoto_contagem.columns = ['tipo_trabalho', 'quantidade']

fig = px.pie(remoto_contagem,
             names='tipo_trabalho',
             values='quantidade',
             title='Proporção dos regimes de trabalho',
          )

fig.show()

In [ ]:
remoto_contagem = df['regime_trabalho'].value_counts().reset_index()
remoto_contagem.columns = ['tipo_trabalho', 'quantidade']

fig = px.pie(remoto_contagem,
             names='tipo_trabalho',
             values='quantidade',
             title='Proporção dos regimes de trabalho',
             hole=0.5
          )

fig.show()

In [ ]:
remoto_contagem = df['regime_trabalho'].value_counts().reset_index()
remoto_contagem.columns = ['tipo_trabalho', 'quantidade']

fig = px.pie(remoto_contagem,
             names='tipo_trabalho',
             values='quantidade',
             title='Proporção dos tipos de trabalho',
             hole=0.5
          )
fig.update_traces(textinfo='percent+label')
fig.show()

In [ ]:
# Função para converter ISO-2 para ISO-3
def iso2_to_iso3(code):
    try:
        return pycountry.countries.get(alpha_2=code).alpha_3
    except:
        return None

# Criar nova coluna com código ISO-3
df['residencia_iso3'] = df['residencia'].apply(iso2_to_iso3)

# Calcular média salarial por país (ISO-3)
df_ds = df[df['cargo'] == 'Data Scientist']
media_ds_pais = df_ds.groupby('residencia_iso3')['salario_dolar'].mean().reset_index()

# Gerar o mapa
fig = px.choropleth(media_ds_pais,
                    locations='residencia_iso3',
                    color='salario_dolar',
                    color_continuous_scale='rdylgn',
                    title='Salário médio de Cientista de Dados por país',
                    labels={'salario_dolar': 'Salário médio (USD)', 'residencia_iso3': 'País'})

fig.show()

In [ ]:
df.to_csv('../data/processed/salarios_final.csv', index=False)

In [ ]:
df[df['residencia'] == 'BR'].value_counts()